<a href="https://colab.research.google.com/github/yeoun9/torchpm/blob/main/examples/example_covariate_searching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install git+https://github.com/yeoun9/torchpm.git
! pip install sympytorch

  Cloning https://github.com/yeoun9/torchpm.git to c:\users\sungwoo\appdata\local\temp\pip-req-build-yrsrxw8_
  Resolved https://github.com/yeoun9/torchpm.git to commit 157fe38386bfb4b92ef29e748837127f5a2c7d6f
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for torchpm: filename=torchpm-0.0.3-py3-none-any.whl size=20945 sha256=0066fc8b94c5edc34afc23cafebb118a87a87f900fcb9bbaed080447150003a8
  Stored in directory: C:\Users\sungwoo\AppData\Local\Temp\pip-ephem-wheel-cache-othw8u5i\wheels\dc\72\dd\d30f04632dad3b07fc5fd9017cf6af44bedce51258901ca564
Successfully built torchpm


  Running command git clone --filter=blob:none --quiet https://github.com/yeoun9/torchpm.git 'C:\Users\sungwoo\AppData\Local\Temp\pip-req-build-yrsrxw8_'


In [2]:
from torchpm import *
from torchpm.data import CSVDataset
from torchpm.parameter import *
from torchpm import covariate
import torch as tc
import numpy as np
import matplotlib.pyplot as plt

# Model Building

In [3]:
dataset_file_path = 'https://raw.githubusercontent.com/yeoun9/torchpm/main/examples/THEO_cov_searching.csv'
dataset_np = np.loadtxt(dataset_file_path, delimiter=',', dtype=np.float32, skiprows=1)
device = tc.device("cuda:0" if tc.cuda.is_available() else "cpu")
column_names = ['ID', 'AMT', 'TIME', 'DV', 'CMT', "MDV", "RATE", 'BWT', 'fixed', 'rand-1+1', 'norm(0,1)', 'BWT-0.5+0.5']
dataset = CSVDataset(dataset_np, column_names, device)

class BasementModel(predfunction.PredictionFunctionByTime) :
    
    def _set_estimated_parameters(self):
        self.theta_0 = Theta(0., 1.5, 10.)
        self.theta_1 = Theta(0., 30., 100.)
        self.theta_2 = Theta(0, 0.08, 1)

        self.eta_0 = Eta()
        self.eta_1 = Eta()
        self.eta_2 = Eta()

        self.eps_0 = Eps()
        self.eps_1 = Eps()

        self.gut_model = linearode.Comp1GutModelFunction()
    
    def _calculate_parameters(self, covariates):
        covariates['k_a'] = self.theta_0()*tc.exp(self.eta_0())
        covariates['v'] = self.theta_1()*tc.exp(self.eta_1())#*para['BWT']/70
        covariates['k_e'] = self.theta_2()*tc.exp(self.eta_2())
        covariates['AMT'] = tc.tensor(320., device=self.dataset.device)

    def _calculate_preds(self, t, p):
        dose = p['AMT'][0]
        k_a = p['k_a']
        v = p['v']
        k_e = p['k_e']
        comps = self.gut_model(t, k_a, k_e, dose)
        return comps[1]/v
        
    def _calculate_error(self, y_pred, p):
        p['v_v'] = p['v'] 
        return y_pred +  y_pred * self.eps_0() + self.eps_1()

dependent_parameter_names = ['k_a', 'v', 'k_e']
dependent_parameter_initial_values = [[0,1.4901,2],[30,32.4667,34],[0,0.08,0.1]]
independent_parameter_names = ['BWT', 'fixed', 'rand-1+1', 'norm(0,1)', 'BWT-0.5+0.5']

searcher = covariate.DeepCovariateSearching(dataset=dataset,
                                BaseModel=BasementModel,
                                dependent_parameter_names=dependent_parameter_names,
                                independent_parameter_names=independent_parameter_names,
                                dependent_parameter_initial_values=dependent_parameter_initial_values,
                                eps_names=['eps_0', 'eps_1'],
                                omega = Omega([0.4397,
                                                0.0575,  0.0198, 
                                                -0.0069,  0.0116,  0.0205], False, requires_grads=True),
                                sigma = Sigma([[0.0177], [0.0762]], [True, True], requires_grads=[True, True]))
result = searcher.run(tolerance_grad=1e-3, tolerance_change=1e-3)

running_time :  1.3690118789672852 	 total_loss: tensor(1765.0817, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  1.9780128002166748 	 total_loss: tensor(817.4667, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  2.5870120525360107 	 total_loss: tensor(442.7851, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  3.197012186050415 	 total_loss: tensor(332.7042, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  3.8000123500823975 	 total_loss: tensor(260.6097, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  4.407013893127441 	 total_loss: tensor(210.3732, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  5.018013954162598 	 total_loss: tensor(191.9969, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  5.632012367248535 	 total_loss: tensor(185.7871, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  6.245012044906616 	 total_loss: tensor(180.1873, device='cuda:0', grad_fn=<AddBackward0>)
running_time :  6.858011960983276 	 total

In [6]:
print('selected covariates \n', result['selected covariates'])

print('========== history ==========')
for record in result['history'] :
    print(record)


selected covariates 
 ['BWT', 'BWT-0.5+0.5']
========== history ==========
{'loss': 73.9505615234375, 'removed covariates': [], 'loss difference': 0.0}
{'loss': 78.52706146240234, 'removed covariates': ['BWT'], 'loss difference': 4.576499938964844}
{'loss': 73.9505615234375, 'removed covariates': [], 'loss difference': -4.576499938964844}
{'loss': 73.314453125, 'removed covariates': ['fixed'], 'loss difference': -0.6361083984375}
{'loss': 75.3486099243164, 'removed covariates': ['fixed', 'rand-1+1'], 'loss difference': 2.0341567993164062}
{'loss': 78.13015747070312, 'removed covariates': ['fixed', 'rand-1+1', 'norm(0,1)'], 'loss difference': 2.7815475463867188}
{'loss': 83.57380676269531, 'removed covariates': ['fixed', 'rand-1+1', 'norm(0,1)', 'BWT-0.5+0.5'], 'loss difference': 5.4436492919921875}
{'loss': 78.13015747070312, 'removed covariates': ['fixed', 'rand-1+1', 'norm(0,1)'], 'loss difference': -5.4436492919921875}
